#SYSC 5906:
## **Room Detection - V1.0**

---
Script to train and validate a **Keras model** to peform room detection based on a processed version of the MIT Indoor Scenes dataset.

###Step 1: Access Drive
Mount the drive with the provided .zip file of code located in it

In [ ]:
#Enter the gdrive
from google.colab import drive
drive.mount('/gdrive',force_remount=True)

Mounted at /gdrive


##Step 2: Setup
Import relevenat libraries, load original dataset into colabs (without the images) and get the folders inside

In [ ]:
#Get tensorFlow built in datasets
!pip install -q tfds-nightly tensorflow tensorflow-datasets
!pip install github-clone

In [ ]:
import tensorflow as tf
import pandas as pd
import pickle
import sklearn as sk
import matplotlib.pyplot as plt
import numpy as np
import tensorflow_datasets as tfds
import keras.api._v2.keras as keras
from tensorflow.keras.utils import to_categorical
from keras.layers import Activation, Dense, Dropout, Input, Flatten, Embedding, Masking, LSTM, Conv2D, MaxPooling2D, Conv1D, MaxPooling1D
from keras.models import Sequential, Model
from keras.optimizers import adam_v2
from keras.engine.input_layer import InputLayer
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from keras.backend import flatten
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler  
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

MODEL_DIRECTORY = '/gdrive/My Drive/Colab Notebooks/SYSC 5906/models/weights/'
#PICKLE_DIRECTORY = '/gdrive/My Drive/Colab Notebooks/SYSC 5906/datasets/mit_indoors/processed/data_subset/7 classes/'
PICKLE_DIRECTORY = '/gdrive/My Drive/Colab Notebooks/SYSC 5906/datasets/mit_indoors/processed/data_labelsOnly/7 classes/'

**Get the datasets:**

In [ ]:
#Import a subset of the MIT dataset
pickledData = open(PICKLE_DIRECTORY+"listOfAllObjLoc_v2.pkl","rb")
dataSet = pickle.load(pickledData)
pickledData.close()

#Import the NYU dataset 
# trainingData = pd.read_csv("/content/sample_data/mnist_train_small.csv")       #TODO: ADD THIS IN

#Import the SUN-RGBD dataset
# trainingData = pd.read_csv("/content/sample_data/mnist_train_small.csv")       #TODO: ADD THIS IN

In [ ]:
#Split the selected dataset into training and validation subsets
training_set, validation_set = train_test_split(dataSet, train_size = 0.8, test_size = 0.2, random_state = 42)

#Splitting the predictors (input data) and target 
#variables (answers/things to classify) as X and Y
x_train = [sublist[0] for sublist in training_set]

x_train_hot=[]
for x_sample in x_train:
    x_hot = to_categorical(x_sample)
    x_train_hot.append(x_hot)

y_train = [sublist[1] for sublist in training_set]
x_val = [sublist[0] for sublist in validation_set]

x_val_hot=[]
for x_sample in x_val:
    x_hot = to_categorical(x_sample)
    x_val_hot.append(x_hot)

y_val = [sublist[1] for sublist in validation_set]


x_train_tensor = tf.constant(x_train_hot)
y_train_tensor = tf.constant(y_train)
x_val_tensor = tf.constant(x_val_hot)
y_val_tensor = tf.constant(y_val)



#Number of classes to detect based on the dataset imported
NUM_CLASSES = 7 #12 for main subset, 7 for more balanced version

##Step 3: Build the Models
Define all models to be trained and tested

##**MODEL 1: STATIC**
Most basic **sequential FNN model** with a hidden dropout and 2 hidden layers

In [ ]:
#MODEL #1
MODEL_ID = 'room_classifier_1.sav'

#Build a model
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(50,50),ragged=True),
    tf.keras.layers.Flatten(), #MODEL DOES NOT WORK WITHOUT THIS FLATTEN!!!!
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(12)
])

#Visualize the model
print(model.summary()) 
plot_model(model, to_file='model.png', show_shapes=True, dpi=100, show_layer_names=True)

#Define a loss function for training the model
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
#This loss is equal to the negative log probability of the true class: 
# The loss is zero if the model is sure of the correct class.
#This untrained model gives probabilities close to random (1/10 for each class),
# so the initial loss should be close to `-tf.math.log(1/10) ~= 2.3`.

#Configure and compile the model
model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 2500)              0         
                                                                 
 dense_2 (Dense)             (None, 128)               320128    
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 12)                1548      
                                                                 
Total params: 321,676
Trainable params: 321,676
Non-trainable params: 0
_________________________________________________________________
None


##**MODEL 2:** 
Basic **sequential FNN model** with a hidden dropout and 3 hidden layers

In [ ]:
#MODEL #2
MODEL_ID = 'room_classifier_2.sav'

#Build a model
model = Sequential()

#Define the model layers
model.add(InputLayer(input_shape=(111,3),ragged=True, name='Input_Layer'))
model.add(tf.keras.layers.Flatten()) #MODEL DOES NOT WORK WITHOUT THIS FLATTEN!!!!
model.add(Dense(795, activation='relu', name='Hidden_Layer_1'))
model.add(Dense(512, activation='relu', name='Hidden_Layer_2'))
model.add(Dense(256, activation='relu', name='Hidden_Layer_3'))
model.add(Dense(128, activation='relu', name='Hidden_Layer_4'))
model.add(Dropout(0.2)) #Dropout hidden layer during training
model.add(Dense(NUM_CLASSES, activation='softmax', name='Output_Layer'))

#Visualize the model
print(model.summary())
plot_model(model, to_file='model.png', show_shapes=True, dpi=100, show_layer_names=True)

#Define a loss function for training the model
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

#Configure and compile the model
model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_13 (Flatten)        (None, 333)               0         
                                                                 
 Hidden_Layer_1 (Dense)      (None, 795)               265530    
                                                                 
 Hidden_Layer_2 (Dense)      (None, 512)               407552    
                                                                 
 Hidden_Layer_3 (Dense)      (None, 256)               131328    
                                                                 
 Hidden_Layer_4 (Dense)      (None, 128)               32896     
                                                                 
 dropout_4 (Dropout)         (None, 128)               0         
                                                                 
 Output_Layer (Dense)        (None, 12)              

##**MODEL 3:** 
Basic **sequential FNN model** with a without any dropout and 3 hidden layers

In [ ]:
#MODEL #3
MODEL_ID = 'room_classifier_3.sav'

#Build a model
model = Sequential()

#Define the model layers
model.add(InputLayer(input_shape=(50,50,1),ragged=True, name='Input_Layer'))

model.add(Conv2D(25, (10,10), activation='relu', input_shape=(50,50)))
model.add(MaxPooling2D((2,2)))
#model.add(tf.keras.layers.Flatten()) #MODEL DOES NOT WORK WITHOUT THIS FLATTEN!!!!
# model.add(Conv1D(25, (5), activation='relu'))
# model.add(MaxPooling2D((2,2)))
# model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(tf.keras.layers.Flatten()) #MODEL DOES NOT WORK WITHOUT THIS FLATTEN!!!!
model.add(Dense(64, activation='relu', name='Hidden_Layer_1'))
model.add(Dense(32, activation='relu', name='Hidden_Layer_2'))
# model.add(Dense(20, activation='relu', name='Hidden_Layer_3'))
model.add(Dense(10, activation = 'relu',name='Hidden_Layer_4'))
model.add(Dense(NUM_CLASSES, activation='softmax', name='Output_Layer'))

#Visualize the model
print(model.summary())
# plot_model(model, to_file='model.png', show_shapes=True, dpi=100, show_layer_names=True)

#Define a loss function for training the model
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
#opt = tf.keras.optimizers.Adam(learning_rate=0.00005,beta_1=0.9,beta_2=0.99,epsilon=1e-7,amsgrad=True)

#Configure and compile the model
model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_7 (Conv2D)           (None, 41, 41, 25)        2525      
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 20, 20, 25)       0         
 g2D)                                                            
                                                                 
 flatten_4 (Flatten)         (None, 10000)             0         
                                                                 
 Hidden_Layer_1 (Dense)      (None, 64)                640064    
                                                                 
 Hidden_Layer_2 (Dense)      (None, 32)                2080      
                                                                 
 Hidden_Layer_4 (Dense)      (None, 10)                330       
                                                     

##**MODEL 4: STATIC** 
Basic **sequential FNN model** without any dropouts and 7 hidden layers

In [ ]:
#MODEL #4
MODEL_ID = 'room_classifier_4.sav'

#Build a model
model = Sequential()

#Define the model layers
model.add(InputLayer(input_shape=(111,3),ragged=True, name='Input_Layer'))
#model.add(Dropout(0.2, input_shape=(60,)))
model.add(tf.keras.layers.Flatten()) #MODEL DOES NOT WORK WITHOUT THIS FLATTEN!!!!
model.add(Dense(2000, activation='relu', name='Hidden_Layer_1'))
model.add(Dense(1000, activation='relu', name='Hidden_Layer_2'))
model.add(Dense(700, activation='relu', name='Hidden_Layer_3'))
model.add(Dense(512, activation='relu', name='Hidden_Layer_4'))
model.add(Dense(256, activation='relu', name='Hidden_Layer_5'))
model.add(Dense(128, activation='relu', name='Hidden_Layer_6'))
model.add(Dense(NUM_CLASSES, activation='softmax', name='Output_Layer'))

#Visualize the model
print(model.summary())
plot_model(model, to_file='model.png', show_shapes=True, dpi=100, show_layer_names=True)

#Define a loss function for training the model
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

#Configure and compile the model
model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])

Model: "sequential_77"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_36 (Flatten)        (None, 333)               0         
                                                                 
 Hidden_Layer_1 (Dense)      (None, 2000)              668000    
                                                                 
 Hidden_Layer_2 (Dense)      (None, 1000)              2001000   
                                                                 
 Hidden_Layer_3 (Dense)      (None, 700)               700700    
                                                                 
 Hidden_Layer_4 (Dense)      (None, 512)               358912    
                                                                 
 Hidden_Layer_5 (Dense)      (None, 256)               131328    
                                                                 
 Hidden_Layer_6 (Dense)      (None, 128)             

##**MODEL 5: STATIC** 
A **functional FNN model** with a without any dropouts and 4 hidden layers

In [ ]:
#MODEL #5
MODEL_ID = 'room_classifier_5.sav'

#Define the model layers
visible_layer = Input(shape=(111,3),ragged=True)
flatten_layer = Flatten()(visible_layer)
hidden1_layer = Dense(1000, activation='relu')(flatten_layer)
hidden2_layer = Dense(100, activation='relu')(hidden1_layer)
hidden3_layer = Dense(50, activation='relu')(hidden2_layer)
output_layer = Dense(NUM_CLASSES, activation='softmax')(hidden3_layer)

#Build a model
model = Model(inputs=visible_layer, outputs=output_layer)

#Visualize the model
print(model.summary())
plot_model(model, to_file='model.png', show_shapes=True, dpi=100, show_layer_names=True)

#Define a loss function for training the model
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

#Configure and compile the model
model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])

##**MODEL 6:** 
A **functional RNN model** with a without hidden dropouts and 4 hidden layers

In [ ]:
#MODEL #6
MODEL_ID = 'room_classifier_6.sav'

#Build a model
model = Sequential()

#Define the model layers
# model.add(Embedding(input_dim=NUM_CLASSES,
#               input_length = len(tensor_x_train),
#               output_dim=100,
#               trainable=False,
#               mask_zero=True))
# model.add(Masking(mask_value=0.0))
model.add(InputLayer(input_shape=(50,50),ragged=False, name='Input_Layer'))
model.add(LSTM(50, return_sequences=True,dropout=0.1, recurrent_dropout=0.1))
#model.add(LSTM(units=50,input_shape=(5,5),return_sequences=True,dropout=0.1, recurrent_dropout=0.1))
model.add(Dense(50, activation='relu'))
model.add(Dense(25, activation='relu'))
model.add(Flatten())
#model.add(Dropout(0.8))
model.add(Dense(NUM_CLASSES, activation='softmax'))
#model.add(Dense(1, activation='softmax'))

# #Build a model
# model = Model(inputs=visible_layer, outputs=output_layer)

#Visualize the model
print(model.summary())
plot_model(model, to_file='model_rnn.png', show_shapes=True, dpi=100, show_layer_names=True)

#Define a loss function for training the model
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
#opt = tf.keras.optimizers.Adam(learning_rate=0.0005,beta_1=0.9,beta_2=0.99,epsilon=1e-7,amsgrad=True)

#Configure and compile the model
model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])

Model: "sequential_28"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_17 (LSTM)              (None, 50, 50)            20200     
                                                                 
 dense_62 (Dense)            (None, 50, 50)            2550      
                                                                 
 dense_63 (Dense)            (None, 50, 25)            1275      
                                                                 
 flatten_18 (Flatten)        (None, 1250)              0         
                                                                 
 dense_64 (Dense)            (None, 7)                 8757      
                                                                 
Total params: 32,782
Trainable params: 32,782
Non-trainable params: 0
_________________________________________________________________
None


##**MODEL 7:** 
A **ensemble model** with two sub-models without any dropout and 3 hidden layers each

In [ ]:
#MODEL #7 --- ENSEMBLE VERSION
MODEL_ID = 'room_classifier_7.sav'

#### Model 1
model_1 = Sequential()
model_1.add(InputLayer(input_shape=(111,3),ragged=True, name='Input_Layer'))
model_1.add(tf.keras.layers.Flatten()) #MODEL DOES NOT WORK WITHOUT THIS FLATTEN!!!!
model_1.add(Dense(512, activation='relu', name='Hidden_Layer_1'))
model_1.add(Dense(256, activation='relu', name='Hidden_Layer_2'))
model_1.add(Dense(128, activation='relu', name='Hidden_Layer_3'))
model_1.add(Dense(NUM_CLASSES, activation='softmax', name='Output_Layer'))

### Model 2
model_2 = Sequential()
model_2.add(InputLayer(input_shape=(111,3),ragged=True, name='Input_Layer'))
model_2.add(tf.keras.layers.Flatten()) #MODEL DOES NOT WORK WITHOUT THIS FLATTEN!!!!
model_2.add(Dense(512, activation='relu', name='Hidden_Layer_1'))
model_2.add(Dense(256, activation='relu', name='Hidden_Layer_2'))
model_2.add(Dense(128, activation='relu', name='Hidden_Layer_3'))
model_2.add(Dense(NUM_CLASSES, activation='softmax', name='Output_Layer'))

#Combine the models in a ensemble
#Build a model
model = [model_1, model_2]
model_input = tf.keras.Input(shape=(125, 125, 3))
model_outputs = [m(model_input) for m in model]
ensemble_output = tf.keras.layers.Average()(model_outputs)
ensemble_model = tf.keras.Model(inputs=model_input, outputs=ensemble_output)

#Visualize the model
print(model.summary())
plot_model(model, to_file='model.png', show_shapes=True, dpi=100, show_layer_names=True)

#Define a loss function for training the model
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

#Configure and compile the model
model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])

##Step 4: Training
Train all models, the models are trained on a portion
of the data that is designated as the training dataset

Train the custom **classifiers**:

In [ ]:
# Create callbacks for training
earlyStop = EarlyStopping(monitor='loss', patience=5)
#modelChkPnt = ModelCheckpoint((MODEL_DIRECTORY+"CHKPNT_"+MODEL_ID), monitor='val_acc', verbose=1, save_best_only=True, mode='max') #TODO: Fix this

callbacks = [earlyStop]

In [ ]:
#Train the model on the dataset
history = model.fit(x_train_tensor, y_train_tensor, epochs=100, 
          callbacks=callbacks, validation_data=(x_val_tensor,y_val_tensor)) #7011, cols in data = 2337
print(history)

#Wrap the model so it returns a probability
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

Epoch 1/100
35/35 [==============================] - 2s 55ms/step - loss: 2.9053 - accuracy: 0.1489 - val_loss: 2.0423 - val_accuracy: 0.2080
Epoch 2/100
35/35 [==============================] - 2s 63ms/step - loss: 2.0141 - accuracy: 0.2320 - val_loss: 2.0488 - val_accuracy: 0.2445
Epoch 3/100
35/35 [==============================] - 2s 51ms/step - loss: 1.9191 - accuracy: 0.2438 - val_loss: 2.1027 - val_accuracy: 0.2445
Epoch 4/100
35/35 [==============================] - 2s 51ms/step - loss: 1.9105 - accuracy: 0.2557 - val_loss: 2.0627 - val_accuracy: 0.2482
Epoch 5/100
35/35 [==============================] - 2s 51ms/step - loss: 1.8696 - accuracy: 0.2712 - val_loss: 2.1383 - val_accuracy: 0.2482
Epoch 6/100
35/35 [==============================] - 2s 52ms/step - loss: 1.8222 - accuracy: 0.2758 - val_loss: 2.0762 - val_accuracy: 0.2555
Epoch 7/100
35/35 [==============================] - 2s 51ms/step - loss: 1.7861 - accuracy: 0.2840 - val_loss: 2.3166 - val_accuracy: 0.2518
Epoch 

##Step 5: Validation
Validate the models on the validation dataset

In [ ]:
#Check the performance of the model on the validation set
model.evaluate(x_val_tensor,  y_val_tensor, verbose=2)
probability_model(x_val_tensor[:5])
y_pred = model.predict(x_val_tensor)

#Determine classification from the outputed probabilities
y_classes = []
for y in y_pred:
    y_classes.append(y.argmax())
print(y_classes)

9/9 - 0s - loss: 4.6968 - accuracy: 0.3066 - 327ms/epoch - 36ms/step
[0, 3, 5, 3, 0, 0, 1, 1, 4, 0, 4, 1, 1, 4, 0, 4, 1, 1, 0, 3, 4, 0, 1, 5, 3, 4, 1, 6, 4, 3, 1, 1, 0, 5, 5, 5, 1, 1, 1, 5, 1, 2, 2, 1, 1, 5, 1, 1, 2, 0, 5, 2, 1, 5, 6, 4, 4, 4, 2, 1, 0, 1, 1, 6, 5, 1, 1, 5, 0, 5, 1, 1, 0, 0, 4, 1, 2, 5, 1, 0, 0, 3, 0, 1, 5, 6, 1, 3, 5, 0, 0, 1, 5, 4, 4, 3, 0, 1, 1, 1, 0, 3, 0, 5, 5, 6, 6, 2, 0, 1, 1, 5, 5, 1, 0, 5, 5, 5, 1, 1, 5, 3, 6, 0, 1, 2, 0, 5, 1, 1, 5, 0, 0, 6, 5, 5, 4, 1, 4, 0, 3, 0, 1, 4, 1, 1, 2, 0, 0, 6, 0, 6, 1, 5, 1, 1, 3, 1, 2, 5, 5, 0, 5, 3, 2, 1, 4, 1, 4, 5, 3, 5, 2, 6, 0, 1, 5, 0, 0, 0, 3, 0, 1, 4, 3, 1, 3, 0, 0, 6, 5, 1, 0, 4, 3, 0, 0, 4, 0, 1, 0, 0, 3, 5, 6, 3, 2, 0, 5, 1, 2, 5, 5, 4, 4, 5, 0, 0, 0, 0, 1, 4, 4, 5, 1, 5, 1, 1, 5, 2, 0, 6, 1, 1, 5, 6, 5, 1, 5, 6, 4, 1, 3, 2, 0, 4, 4, 1, 2, 4, 0, 0, 1, 3, 1, 4, 1, 5, 3, 3, 1, 0, 0, 5, 4, 6, 2, 5, 5, 0, 3, 4, 4, 1]


##Step 6: Results
Validate the models, determines the accuracy of their predictions on
the validation dataset

In [ ]:
#Setup confusion matrix figure
plt.rcParams['figure.figsize'] = (100,100)

#Plot the accuracy (confusion matrix)
ConfusionMatrixDisplay.from_predictions(y_val, y_classes)
plt.show()

#Basic version of the CF
print(tf.math.confusion_matrix(y_val,y_classes))

In [ ]:
#Determine metrics
# 'micro': Calculate metrics globally by counting the total true positives, false negatives and false positives.
# 'macro': Calculate metrics for each label, and find their unweighted mean. This does not take label imbalance into account.
print("Accuracy on validation data: %f" % sk.metrics.accuracy_score(y_val, y_classes))
print("Balanced accuracy on validation data: %f" % sk.metrics.balanced_accuracy_score(y_val, y_classes))
print("Recall score on validation data: %f" % sk.metrics.recall_score(y_val, y_classes, average='macro', zero_division=0))
print("Precision score on validation data: %f" % sk.metrics.precision_score(y_val, y_classes, average='macro', zero_division=0))
print("F1 score on validation data: %f" % sk.metrics.f1_score(y_val, y_classes, average='macro'))

Accuracy on validation data: 0.306569
Balanced accuracy on validation data: 0.281212
Recall score on validation data: 0.281212
Precision score on validation data: 0.283601
F1 score on validation data: 0.276325


##Step 7: Exporting & Deploying
Save the model as a pickle

In [ ]:
#Save the current model
model.save(MODEL_DIRECTORY+MODEL_ID)